Decribe the structure of order data


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/B2B_Order_Data.csv')

# Preview the first few rows
print(df.head())

# Get summary statistics and data types
print(df.info())
print(df.describe(include='all'))
print(df.dtypes)


  Supplier_ID  Order_Date Shipping_Date Expected_Delivery_Date Product_Code  \
0        SUP1  2025-03-24    2025-03-30             2025-03-31         P001   
1        SUP1  2025-04-22    2025-04-28             2025-05-04         P003   
2        SUP1  2025-07-22    2025-07-29             2025-07-30         P003   
3        SUP1  2024-10-04    2024-10-08             2024-10-18         P005   
4        SUP1  2024-10-19    2024-11-02             2024-11-08         P005   

       Product_Name  Per_Unit_Cost  Quantity  Total_Cost  
0         Inverters          38.49        85     3271.65  
1  Onboard Chargers         113.46       142    16111.32  
2  Onboard Chargers         119.66       140    16752.40  
3            Struts         349.00       184    64216.00  
4            Struts         372.14        48    17862.72  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------ 

Clean the dataset to remove duplicate rows or any rows with missing data


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/B2B_Order_Data.csv')

# Remove duplicate rows
df_no_duplicates = df.drop_duplicates()

# Remove rows with any missing values
df_cleaned = df_no_duplicates.dropna()

# Optional: Save the cleaned dataset to a new CSV file
cleaned_file_path = '/content/Cleaned_B2B_Order_Data.csv'
df_cleaned.to_csv(cleaned_file_path, index=False)


print(f"Cleaned dataset saved to {cleaned_file_path}")

Cleaned dataset saved to /content/Cleaned_B2B_Order_Data.csv


Compute the average time to delivery by supplier

In [ ]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv('/content/Cleaned_B2B_Order_Data.csv')

# Convert date columns to datetime
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Expected_Delivery_Date'] = pd.to_datetime(df['Expected_Delivery_Date'])

# Calculate lead time in days
df['lead_time_to_delivery'] = (df['Expected_Delivery_Date'] - df['Order_Date']).dt.days

# Save the updated dataset
updated_file_path = '/content/Updated_B2B_Order_Data.csv'
df.to_csv(updated_file_path, index=False)

print(f"Updated dataset with lead time to delivery saved to {updated_file_path}\n\n")






def average_lead_time_by_supplier(csv_file):
    # Load the dataset
    df = pd.read_csv(csv_file)

    # Group by Supplier_ID and calculate the mean lead_time_to_delivery
    avg_lead_time = df.groupby('Supplier_ID')['lead_time_to_delivery'].mean()

    # Return as a DataFrame for better readability
    return avg_lead_time.reset_index().rename(columns={'lead_time_to_delivery': 'average_lead_time'})



result = average_lead_time_by_supplier('/content/Updated_B2B_Order_Data.csv')
print(result)


Updated dataset with lead time to delivery saved to /content/Updated_B2B_Order_Data.csv


  Supplier_ID  average_lead_time
0        SUP1          13.718935
1        SUP2          13.079646
2        SUP3          13.332288


Now create an Inventory Management Simulation

*   Randomly select demand from normal dist (mean - 50, std - 10)
*   Holding cost is $10 per unit per day

*   Shortage cost is $50 per unit per day
*   For each supplier, use their average days to delivery as the order lead time
*   For each supplier, run a 180-day simulation and calculate the inventory management costs for that supplier








In [ ]:
import numpy as np
import pandas as pd

# Parameters
num_simulations = 100
num_days = 180
mean_demand = 50
std_dev_demand = 10
holding_cost_per_unit_per_day = 10
shortage_cost_per_unit_per_day = 50
delivery_times = {
    'Supplier 1': 13.718935,
    'Supplier 2': 13.079646,
    'Supplier 3': 13.332288
}

# Function to run a single simulation
def run_simulation(delivery_time):
    total_holding_cost = 0
    total_shortage_cost = 0
    inventory_level = 0

    daily_demand = np.random.normal(mean_demand, std_dev_demand, num_days)

    for day in range(num_days):
        demand = daily_demand[day]
        inventory_level -= demand

        # If inventory is negative, calculate shortage cost
        if inventory_level < 0:
            total_shortage_cost += abs(inventory_level) * shortage_cost_per_unit_per_day
            inventory_level = 0  # Reset inventory level to 0 after shortage

        # Calculate holding cost
        total_holding_cost += inventory_level * holding_cost_per_unit_per_day

        # Simulate delivery after the specified lead time
        if day % delivery_time == 0:
            inventory_level += mean_demand * delivery_time  # Replenish inventory

    return total_holding_cost, total_shortage_cost

# Running simulations for each supplier
results = {'Supplier': [], 'Simulation': [], 'Total_Holding_Cost': [], 'Total_Shortage_Cost': []}

for supplier, delivery_time in delivery_times.items():
    for sim in range(num_simulations):
        holding_cost, shortage_cost = run_simulation(delivery_time)
        results['Supplier'].append(supplier)
        results['Simulation'].append(sim + 1)
        results['Total_Holding_Cost'].append(holding_cost)
        results['Total_Shortage_Cost'].append(shortage_cost)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display results
print(results_df.head())

# Save results to a CSV file
results_df.to_csv('/content/Inventory_Management_Simulation_Results.csv', index=False)



     Supplier  Simulation  Total_Holding_Cost  Total_Shortage_Cost
0  Supplier 1           1        42167.999628        421874.442836
1  Supplier 1           2        43980.525591        422231.381780
2  Supplier 1           3        42177.829381        422858.358507
3  Supplier 1           4        37371.658762        415668.890016
4  Supplier 1           5        43674.984366        411532.563070


Calculate average total inventory cost by supplier

In [ ]:
# Calculate total inventory cost as sum of holding and shortage costs
results_df['Total_Inventory_Cost'] = results_df['Total_Holding_Cost'] + results_df['Total_Shortage_Cost']

# Calculate average total inventory cost by supplier
average_cost_by_supplier = (
    results_df.groupby('Supplier')['Total_Inventory_Cost']
    .mean()
    .reset_index()
    .rename(columns={'Total_Inventory_Cost': 'Average_Total_Inventory_Cost'})
)

# Display the result
print(average_cost_by_supplier)

# Save to a new CSV file
average_cost_by_supplier.to_csv('/content/Average_Inventory_Cost_By_Supplier.csv', index=False)


     Supplier  Average_Total_Inventory_Cost
0  Supplier 1                 458747.959091
1  Supplier 2                 457215.913816
2  Supplier 3                 458067.039122
